In [3]:
import sys
import os
from mfm.MFM import MFM
import pandas as pd 
import numpy as np
from pymongo import MongoClient

MONGO_CONNECTION_STRING = "mongodb://localhost:27017/"
DB_NAME = "barra_financial_data"

client = MongoClient(MONGO_CONNECTION_STRING)
db = client[DB_NAME] # <--- 获取数据库连接对象

def load_collection_to_df(db, collection_name: str, query: dict, projection: dict) -> pd.DataFrame:
    """加载经过筛选和投影的集合数据。"""
    print(f"正在从 '{collection_name}' 加载数据...")
    collection = db[collection_name]
    cursor = collection.find(query, projection)
    df = pd.DataFrame(list(cursor))
    print(f"-> 成功加载 {len(df):,} 行数据。")
    return df

In [4]:
####导入数据

data = load_collection_to_df(db, collection_name='barra_factors', query={}, projection={'_id':0})
naidx = 1*np.sum(pd.isna(data), axis = 1)>0
data = data[~naidx]
data.index = range(len(data))


####行业数据
industry_info = load_collection_to_df(db, collection_name='sw_industry_info_for_factors', query={}, projection={'_id':0})
industry = np.array([1*(data.industry.values == x) for x in industry_info.code.values]).T
industry = pd.DataFrame(industry, columns = list(industry_info.industry_names.values))
data = pd.concat([data.iloc[:,:4], industry, data.iloc[:,5:]], axis = 1)

正在从 'barra_factors' 加载数据...
-> 成功加载 406,866 行数据。
正在从 'sw_industry_info_for_factors' 加载数据...
-> 成功加载 28 行数据。


In [16]:
data

,date,stocknames,capital,ret,银行,房地产,通信,电子,机械设备,非银金融,...,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage
0,2020/04/09,000001.SZ,24722928.0,0.003925,1,0,0,0,0,0,...,17.023251,1.149444,-0.003281,-0.008897,4.738475,1.104362,-0.185946,0.114043,0.106625,31.351466
1,2020/04/10,000001.SZ,24819958.0,-0.015637,1,0,0,0,0,0,...,17.027166,1.145186,-0.003311,-0.008692,4.146654,1.100110,-0.164237,0.113598,0.106625,31.351466
2,2020/04/13,000001.SZ,24431842.0,0.021446,1,0,0,0,0,0,...,17.011406,1.146689,-0.003258,-0.008747,5.139823,1.117568,-0.164616,0.115401,0.106625,31.351466
3,2020/04/14,000001.SZ,24955798.0,0.000778,1,0,0,0,0,0,...,17.032625,1.149145,-0.003410,-0.008975,4.267335,1.094092,-0.170491,0.112979,0.106625,31.351466
4,2020/04/15,000001.SZ,24975204.0,-0.014763,1,0,0,0,0,0,...,17.033403,1.144425,-0.004388,-0.008786,4.254021,1.093255,-0.170998,0.112891,0.106625,31.351466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385489,2025/10/15,688981.SH,25906332.0,-0.017444,0,0,0,1,0,0,...,18.456520,1.829026,0.000386,0.041162,-69.031829,0.145722,2.028818,191.268359,0.025860,229.280960
385490,2025/10/16,688981.SH,25454432.0,-0.041791,0,0,0,1,0,0,...,18.438923,1.826346,0.000405,0.047946,-65.226334,0.148309,2.005971,194.664011,0.025860,233.137985
385491,2025/10/17,688981.SH,24390664.0,0.005329,0,0,0,1,0,0,...,18.396233,1.831553,0.000065,0.051494,-61.043273,0.154777,2.021167,203.154018,0.025860,242.781586
385492,2025/10/20,688981.SH,24520636.0,0.037185,0,0,0,1,0,0,...,18.401548,1.829165,0.000224,0.051921,-60.538863,0.153955,2.004551,202.077210,0.025860,241.558457


In [15]:
model = MFM(data, 28, 10)
(factor_ret, specific_ret, R2) = model.reg_by_time()
nw_cov_ls = model.Newey_West_by_time(q = 2, tao = 252)                 #Newey_West调整
er_cov_ls = model.eigen_risk_adj_by_time(M = 100, scale_coef = 1.4)    #特征风险调整
vr_cov_ls, lamb = model.vol_regime_adj_by_time(tao = 42) 

===================================逐时间点进行横截面多因子回归===================================
Cross Section Regression, Date: 2020/04/09, 254 Stocks, 28 Industry Facotrs, 10 Style Facotrs

/opt/anaconda3/envs/tf/lib/python3.9/site-packages/mfm/CrossSection.py:75: RuntimeWarning: divide by zero encountered in matmul
  factors_tran = factors @ R
/opt/anaconda3/envs/tf/lib/python3.9/site-packages/mfm/CrossSection.py:75: RuntimeWarning: overflow encountered in matmul
  factors_tran = factors @ R
/opt/anaconda3/envs/tf/lib/python3.9/site-packages/mfm/CrossSection.py:75: RuntimeWarning: invalid value encountered in matmul
  factors_tran = factors @ R
/opt/anaconda3/envs/tf/lib/python3.9/site-packages/mfm/CrossSection.py:76: RuntimeWarning: divide by zero encountered in matmul
  pure_factor_portfolio_weight = R @ np.linalg.inv(factors_tran.T @ W @ factors_tran) @ factors_tran.T @ W  #纯因子组合权重
/opt/anaconda3/envs/tf/lib/python3.9/site-packages/mfm/CrossSection.py:76: RuntimeWarning: overflow encountered in matmul
  pure_factor_portfolio_weight = R @ np.linalg.inv(factors_tran.T @ W @ factors_tran) @ factors_tran.T @ W  #纯因子组合权重
/opt/anaconda3/envs/tf/lib/python3.9/site-packages/m

LinAlgError: Singular matrix